In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import os
import json
import pickle
import requests

In [2]:
movie_data = pd.read_csv("./final_movie_data.csv")
movie_data.head()

,title,genres,director_name,actor1_name,actor2_name,actor3_name,comb
0,pirates of the caribbean: dead men tell no tales,Adventure Action Fantasy Comedy,Joachim Rønning Espen Sandberg,Johnny Depp,Javier Bardem,Geoffrey Rush,Joachim Rønning Espen Sandberg Johnny Depp Jav...
1,justice league,Action Adventure Fantasy Sci-Fi,Zack Snyder,Ben Affleck,Henry Cavill,Gal Gadot,Zack Snyder Ben Affleck Henry Cavill Gal Gadot...
2,thor: ragnarok,Action Adventure Fantasy Sci-Fi,Taika Waititi,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Taika Waititi Chris Hemsworth Tom Hiddleston C...
3,guardians of the galaxy vol. 2,Action Adventure Comedy Sci-Fi,James Gunn,Chris Pratt,Zoe Saldana,Dave Bautista,James Gunn Chris Pratt Zoe Saldana Dave Bautis...
4,the king's daughter,Fantasy Action Adventure,Sean McNamara,Pierce Brosnan,William Hurt,Benjamin Walker,Sean McNamara Pierce Brosnan William Hurt Benj...


In [3]:
def create_similarity():
    movie_data = pd.read_csv('final_movie_data.csv')
    # creating a count matrix
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(movie_data['comb'])
    # creating a similarity score matrix
    similarity = cosine_similarity(count_matrix)
    return movie_data,similarity

In [4]:
def convert(val):
    # isinstance(val, np.generic) will return True if val is an array scalar object.
    if isinstance(val, np.generic):
        return val.item()
    raise TypeError

In [5]:
from tmdbv3api import TMDb
from tmdbv3api import Movie

tmdb = TMDb()
TMDB_API_KEY = "75f49154067c35aa93fcb726bdcc2adb"
tmdb.api_key = TMDB_API_KEY

In [6]:
def movie_search_engine(movie_name, TMDB_API_KEY = TMDB_API_KEY):
    MOVIE_SEARCH_LINK = "https://api.themoviedb.org/3/movie/"
    BASE_IMAGE_URL = "https://image.tmdb.org/t/p/original"
    tmdb_movie = Movie()
    search_result = tmdb_movie.search(movie_name.strip())
    movie_id = search_result[0]["id"]
    response = requests.get(MOVIE_SEARCH_LINK+"{}?api_key={}".format(movie_id, tmdb.api_key))
    json_response = response.json()
    
    movie_data = {}
    movie_data["title"] = json_response["original_title"] if json_response["original_title"] and json_response["original_title"].lower()==movie_name else json_response["title"]
    movie_data["poster"] = BASE_IMAGE_URL + json_response["poster_path"]
    movie_data["overview"] = json_response["overview"]
    genres = []
    for genre in json_response["genres"]:
        genres.append(genre["name"])
    movie_data["genres"] = "*".join(genres)
    movie_data["rating"] = json_response["vote_average"]
    movie_data["total_votes"] = json_response["vote_count"]
    movie_data["tag_line"] = json_response["tagline"]
    movie_data["status"] = json_response["status"]
    return movie_data

In [10]:
def movie_recommender_engine(movie_name, n_top_recommendations=10):
    movie_name = movie_name.strip().lower()
    data, similarity_matrix = create_similarity()
    if movie_name not in data["title"].unique():
        return json.dumps({"error": "Sorry! Movie is not in our database. Please check the spelling or try with another movie name"}, default=convert)
    else:
        index = data.loc[data["title"] == movie_name].index[0]
        matrix = list(enumerate(similarity_matrix[index]))
        matrix = sorted(matrix, key=lambda x:x[1], reverse=True)
        recommended_indexes = [index for (index, similarity) in matrix[1:n_top_recommendations+1]]
        recommended_movies = {}
        for i, index in enumerate(recommended_indexes):
            r_movie_name = data["title"][index]
            movie_data = movie_search_engine(r_movie_name)
            recommended_movies["movie"+str(i+1)] = movie_data
        recommended_movies = json.dumps(recommended_movies, default = convert)
        return recommended_movies

In [11]:
movie_recommender_engine("iron man", 10)

'{"movie1": {"title": "Iron Man 2", "poster": "https://image.tmdb.org/t/p/original/6WBeq4fCfn7AN0o21W9qNcRF2l9.jpg", "overview": "With the world now aware of his dual life as the armored superhero Iron Man, billionaire inventor Tony Stark faces pressure from the government, the press and the public to share his technology with the military. Unwilling to let go of his invention, Stark, with Pepper Potts and James \'Rhodey\' Rhodes at his side, must forge new alliances \\u2013 and confront powerful enemies.", "genres": "Adventure*Action*Science Fiction", "rating": 6.8, "total_votes": 17947, "tag_line": "It\'s not the armor that makes the hero, but the man inside.", "status": "Released"}, "movie2": {"title": "Iron Man 3", "poster": "https://image.tmdb.org/t/p/original/qhPtAc1TKbMPqNvcdXSOn9Bn7hZ.jpg", "overview": "When Tony Stark\'s world is torn apart by a formidable terrorist called the Mandarin, he starts an odyssey of rebuilding and retribution.", "genres": "Action*Adventure*Science F